In [ ]:
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.express as px

import pandas as pd

from typing import List, Any
from trino import dbapi


pyo.init_notebook_mode()


# Connection configuration to TrinoDB.
conn = dbapi.connect(
    host='51.103.220.68',
    port=8080,
    user='trino',
    catalog='mongodb',
    schema='ghdb',
)

# Cursor to make SQL query.
cursor = conn.cursor()

In [ ]:
def fetch(query: str, columns: List[str]) -> pd.DataFrame:
    cursor.execute(query)
    results = cursor.fetchall()

    return pd.DataFrame(results, columns=columns)

In [ ]:
# Return the number of user type.
# [[User, 60], [Organization, 1]]
dataframe = fetch('SELECT type, COUNT() AS count FROM users GROUP BY type', ['Type', 'Value'])

fig = px.pie(dataframe, names='Type', values='Value')
fig.show()

In [ ]:
dataframe = fetch(
    'SELECT language, COUNT(*) AS count FROM repos WHERE language IS NOT NULL GROUP BY language ORDER BY count DESC LIMIT 5', 
    columns=['Language', 'Value']
)

fig = px.pie(dataframe, names='Language', values='Value')
fig.show()

In [ ]:
dataframe = fetch(
    'SELECT u.type, COUNT() AS count FROM users AS u JOIN repos AS r ON u.login = r.login GROUP BY u.type', 
    columns=['Type', 'Value']
)

fig = px.histogram(dataframe, x="Type", y="Value")
fig.show()

In [ ]:
dataframe = fetch(
    'SELECT users.login, COUNT() as repo_count FROM users JOIN repos ON users.login = repos.login GROUP BY users.login ORDER BY repo_count DESC LIMIT 40', 
    columns=['User', 'Repo']
)

fig = px.bar(dataframe, x='User', y='Repo', text_auto='.2s', title="User and number of repositories.")
fig.show()

In [ ]:
dataframe = fetch(
    'SELECT u.login, SUM(r.stargazers_count) AS avg_stars FROM users u JOIN repos r ON u.login = r.login GROUP BY u.login ORDER BY avg_stars DESC LIMIT 10', 
    columns=['User', 'Star']
)

fig = px.pie(dataframe, names='User', values='Star')
fig.show()


dataframe = fetch(
    'SELECT u.login, AVG(r.stargazers_count) AS avg_stars FROM users u JOIN repos r ON u.login = r.login WHERE r.stargazers_count >= 1 GROUP BY u.login ORDER BY avg_stars DESC', 
    columns=['User', 'Star']
)

fig = px.bar(dataframe, x='User', y='Star', text_auto='.2s', title="AVG of start for each users.")
fig.show()

In [ ]:
dataframe = fetch(
    'SELECT u.login, AVG(r.stargazers_count) AS avg_stars FROM users u JOIN repos r ON u.login = r.login WHERE r.stargazers_count >= 1 AND u.login <> \'flutter\' GROUP BY u.login ORDER BY avg_stars DESC LIMIT 10', 
    columns=['User', 'Star']
)

fig = px.pie(dataframe, names='User', values='Star')
fig.show()

In [ ]:
dataframe = fetch(
    'SELECT language, COUNT(*) AS count FROM repos WHERE language IS NOT NULL GROUP BY language ORDER BY count DESC LIMIT 15', 
    columns=['Language', 'Value']
)

fig = px.bar(dataframe, x='Language', y='Value', text_auto='.2s')
fig.show()

In [ ]:
dataframe = fetch(
    'SELECT license, COUNT(*) AS count FROM repos WHERE license IS NOT NULL GROUP BY license ORDER BY count DESC', 
    columns=['License', 'Value']
)

fig = px.bar(dataframe, x='License', y='Value', text_auto='.2s')
fig.show()